# Update maps for functional groups of the IUCN Global Ecosystem typology

Scripts by José R. Ferrer-Paris

The scripts described in this document are used to:

- Check map information in the database
- Read json file with map information for each EFG
- Upload maps to mapbox

## Set-up
Load all the libraries we will need in this script:

In [1]:
import os
from pathlib import Path
from configparser import ConfigParser
import psycopg2
import pandas as pd
import json

Read configuration parameters for the connection to the current version of the database:

In [2]:
filename = Path(os.path.expanduser('~')) / ".database.ini"
section = 'psqlaws'

parser = ConfigParser()
parser.read(filename)
db = {}
if parser.has_section(section):
    params = parser.items(section)
    for param in params:
        db[param[0]] = param[1]
else:
    raise Exception('Section {0} not found in the {1} file'.format(section, filename))

params = db

In [16]:
conn = psycopg2.connect(**params)
cur = conn.cursor()

## List of valid maps

In [24]:
qry="""
SELECT code, map_code, map_version, map_file, file_description, file_comments,status
FROM map_files
LEFT JOIN map_metadata
USING(map_code,map_version)
WHERE map_file ilike '%mapbox%'
ORDER BY code
""";
cur.execute(qry)
list_maps=cur.fetchall()

In [48]:
qry="""
SELECT code, map_code, map_version,map_type, status, update
FROM  map_metadata
WHERE status ilike 'valid'
ORDER BY code
""";
cur.execute(qry)
valid_maps=cur.fetchall()

In [49]:

df=pd.DataFrame(valid_maps,columns=['code','map_code', 'map_version', 'map_type','status', 'update'])
df

,code,map_code,map_version,map_type,status,update
0,F1.1,F1.1.web.mix,v2.0,Web navigation,valid,2020-11-08 00:40:25.604
1,F1.1,F1.1.IM.mix,v2.0,Indicative Map,valid,2020-07-11 00:00:00.000
2,F1.2,F1.2.IM.grid,v1.0,Indicative Map,valid,2020-07-09 00:00:00.000
3,F1.2,F1.2.web.map,v1.0,Web navigation,valid,2020-11-08 00:39:29.066
4,F1.3,F1.3.IM.grid,v1.0,Indicative Map,valid,2020-07-09 00:00:00.000
...,...,...,...,...,...,...
206,TF1.5,TF1.5.web.map,v2.0,Web navigation,valid,2020-11-06 20:25:37.000
207,TF1.6,TF1.6.IM.orig,v1.0,Indicative Map,valid,2020-07-02 00:00:00.000
208,TF1.6,TF1.6.web.orig,v1.0,Web navigation,valid,2020-10-19 23:59:50.829
209,TF1.7,TF1.7.IM.orig,v1.0,Indicative Map,valid,2020-07-06 00:00:00.000


In [50]:
df1=df.query("code=='T7.5'")
df1

,code,map_code,map_version,map_type,status,update
196,T7.5,T7.5.WM.nwx,v2.0,Indicative Map,valid,2021-11-19 00:19:08


In [28]:
df=pd.DataFrame(list_maps,columns=['code','map_code', 'map_version','map_file', 'description', 'comment', 'status'])
df

,code,map_code,map_version,map_file,description,comment,status
0,F1.1,F1.1.web.mix,v1.0,https://studio.mapbox.com/tilesets/jrfep.F1_1_...,raster map in repository,GeoTIFF version uploaded to Mapbox,requires review
1,F1.1,F1.1.web.mix,v2.0,https://studio.mapbox.com/tilesets/jrfep.F1_1_...,raster map in repository,GeoTIFF version uploaded to Mapbox,valid
2,F1.2,F1.2.web.map,v1.0,https://studio.mapbox.com/tilesets/jrfep.F1_2_...,raster map in repository,GeoTIFF version uploaded to Mapbox,valid
3,F1.2,F1.2.web.grid,v1.0,https://studio.mapbox.com/tilesets/jrfep.F1_2_...,raster map in repository,GeoTIFF version uploaded to Mapbox,duplicated
4,F1.2,F1.2.web.mix,v1.0,https://studio.mapbox.com/tilesets/jrfep.F1_2_...,raster map in repository,GeoTIFF version uploaded to Mapbox,requires review
...,...,...,...,...,...,...,...
116,T7.5,T7.5.web.orig,v2.0,https://studio.mapbox.com/tilesets/jrfep.T7_5_...,raster map in repository,GeoTIFF version uploaded to Mapbox,requires review
117,T7.5,T7.5.web.mix,v1.0,https://studio.mapbox.com/tilesets/jrfep.T7_5_...,raster map in repository,GeoTIFF version uploaded to Mapbox,superceeded
118,TF1.1,TF1.1.web.map,v1.0,https://studio.mapbox.com/tilesets/jrfep.TF1_1...,raster map in repository,GeoTIFF version uploaded to Mapbox,valid
119,TF1.4,TF1.4.web.map,v1.0,https://studio.mapbox.com/tilesets/jrfep.TF1_4...,raster map in repository,GeoTIFF version uploaded to Mapbox,valid


In [43]:
df1=df.query("code=='T7.5'")
df1

,code,map_code,map_version,map_file,description,comment,status
116,T7.5,T7.5.web.orig,v2.0,https://studio.mapbox.com/tilesets/jrfep.T7_5_...,raster map in repository,GeoTIFF version uploaded to Mapbox,requires review
117,T7.5,T7.5.web.mix,v1.0,https://studio.mapbox.com/tilesets/jrfep.T7_5_...,raster map in repository,GeoTIFF version uploaded to Mapbox,superceeded


In [44]:
df1[['map_version','map_file']].values

array([['v2.0', 'https://studio.mapbox.com/tilesets/jrfep.T7_5_web_orig'],
       ['v1.0', 'https://studio.mapbox.com/tilesets/jrfep.T7_5_web_mix']],
      dtype=object)

Read json file

In [39]:
#config.json <- read_json(sprintf("%s/data/config/groups.json", Sys.getenv("WEBDATAREPO")))
web_config = Path(os.environ['WEBDATAREPO']) / 'data/config/groups.json'
with open(web_config) as f:
    config_json = json.load(f)



In [42]:
config_json[100]

{'id': 'T7.5',
 'path': 't_7_5',
 'biome': 'T7',
 'layer': {'source': 'mapbox',
  'type': 'raster',
  'tileset': 'jrfep.T7_5_web_mix',
  'extent': {'N': '90', 'S': '90', 'W': '180', 'E': '180'}},
 'title': {'en': 'Derived semi-natural pastures and old fields'},
 'image': {'credit': {'en': 'David Keith'},
  'caption': {'en': 'Semi-natural grassland, South Downs, England'}}}

ModuleNotFoundError: No module named 'gdal'

## That's it

Let's close the connection:

In [15]:

cur.close()
        
if conn is not None:
    conn.close()
    print('Database connection closed.')

Database connection closed.
